In [1]:
#pip install python-docx
#pip install docx

In [2]:
import docx
import gensim.summarization
import os 
import tkinter as tk
from tkinter.filedialog import askopenfilename
from IPython.display import Markdown, display
import nltk   
from bs4 import BeautifulSoup
import requests

from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

In [3]:
def openfile():
    # Hide the additional dialog box
    root = tk.Tk()
    root.withdraw()
    root.wm_attributes('-topmost', 1)

    filename = askopenfilename(initialdir=os.getcwd(), title="Choose Input File", filetypes = (("Word Documents","*.docx"),("Text Files","*.txt"),("PDF Documents","*.pdf")))

    return filename

In [4]:
def getText(filename):
    extension = os.path.splitext(filename)[1][1:]
    if extension == "txt":
        f = open(filename, "r")
        fullText = f.read()
        return fullText

    if extension == "docx":
        doc = docx.Document(filename)
        fullText = []
        for para in doc.paragraphs:
            fullText.append(para.text)
        return '\n'.join(fullText)

    if extension == "pdf":
        output_string = StringIO()
        with open(filename, 'rb') as in_file:
            parser = PDFParser(in_file)
            doc = PDFDocument(parser)
            rsrcmgr = PDFResourceManager()
            device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
            interpreter = PDFPageInterpreter(rsrcmgr, device)
            for page in PDFPage.create_pages(doc):
                interpreter.process_page(page)

        return output_string.getvalue()
    
    return ""   

In [5]:
def summarise(text, word_count=300):
    #text = getText(filename)
    if text == "":
        return ""
    striptext = text.replace('\n\n', ' ')
    striptext = striptext.replace('\n', ' ')
    striptext = striptext.replace('\t', ' ')
    striptext = striptext.replace('i.e.', '[[ie]]')
    
    summary = gensim.summarization.summarize(striptext, word_count=word_count)
    
    return summary.replace(".", ".\n").replace('[[ie]]', 'i.e.')    

In [6]:
def printmd(string):
    display(Markdown(string))

In [7]:
def openLocalFileAndSummarize():
    file_name = openfile()
    if file_name != "": 
        printmd(f"<b>Executive Summary for {os.path.basename(file_name)}\n</b>")
        print(summarise(getText(file_name)))

In [9]:
openLocalFileAndSummarize()

<b>Executive Summary for CyberSecurityForDummies.pdf
</b>

Cybersecurity Means Different Things to Different Folks Cybersecurity Is a Constantly Moving Target Looking at the Risks That Cybersecurity Mitigates Chapter 2: Getting to Know Common Cyberattacks Attacks That Inflict Damage Impersonation Interception Data Theft Malware Poisoned Web Service Attacks Network Infrastructure Poisoning Malvertising Exploiting Maintenance Difficulties Advanced Attacks Chapter 3: Bad Guys and Accidental Bad Guys: The Folks You Must Defend Against Bad Guys and Good Guys Are Relative Terms Bad Guys Up to No Good Cyberattackers and Their Colored Hats Monetizing Their Actions Dealing with Nonmalicious Threats Defending against These Attackers Addressing Risks through Various Methods Part 2: Improving Your Own Personal Security Chapter 4: Evaluating Your Current Cybersecurity Posture Identifying Ways You May Be Less than Secure Identifying Risks Protecting against Risks Evaluating Your Current Security Measures Privacy 101 Banking Online Safely Safely Using Smart